# Hiztegi baten inplementazioa

## Kolisioen ebazpena I : Taula berdimentsionatu

* $g$ gako bati taulan dagokion posizioa: $i \; = \; hash(g)\; \% \; N$
   * $i$ posizioan `(gako,balio)` bikotea gordeko da
   * Hutsik dagoen gelaxkan, `None` egongo da   
* Kolisio bat ematen den bakoitzean, taularen tamaina handitu.
* $LF \; \ll \; 1$


#### 1 - Hiztegi berri bat sortzeko funtzioa, `dict()`-en baliokidea

In [1]:
def h_new(n=7):
    return [None] * n

#### 2 - Sortu hiztegi bat eta *eskuz* gorde ondokoak: `3:'hiru'` eta `'lau':4`

In [2]:
h = h_new()
h[hash(3)%len(h)] = (3,'hiru')
h[hash('lau')%len(h)] = ('lau',4)
print(h)

#### 3 - Hiztegi baten `(gako,balio)` bikote zerrenda bueltatuko duen funtzioa, `dict.items()`-ren baliokidea

In [3]:
def h_items(h):
    z = []
    for x in h :
        if x :
            z.append(x)
    return z

In [4]:
print('h zerrendako elementuak:',*h)
print('h hiztegiko elementuak:',*h_items(h))

h zerrendako elementuak: None None ('lau', 4) (3, 'hiru') None None None
h hiztegiko elementuak: ('lau', 4) (3, 'hiru')


#### 4 - Hiztegi baten tamaina bueltatuko duen funtzioa, `len()`-ren baliokidea

In [5]:
def h_len(h):
    n = 0
    for x in h :
        if x :
            n += 1
    return n

def h_len2(h):
    return len(h_items(h))

In [6]:
print(f'h zerrendaren tamaina: {len(h)}')
print(f'h hiztegiaren tamaina: {h_len(h)}')
print(f'h hiztegiaren tamaina: {h_len2(h)}')

h zerrendaren tamaina: 7
h hiztegiaren tamaina: 2
h hiztegiaren tamaina: 2


#### 5 - Hiztegi batetik karaktere kate bat sortzeko funtzioa, `str()`-ren baliokidea

In [7]:
def h_str(h):
    z = []
    for k,v in h_items(h) :
        z.append(repr(k) + ':' + repr(v))
    return '{ ' + ' , '.join(z) + ' }'

In [8]:
print(str(h))
print(h_str(h))

[None, None, ('lau', 4), (3, 'hiru'), None, None, None]
{ 'lau':4 , 3:'hiru' }


#### 6 - Gako bati dagokion balioa kontsultatzeko funtzioa, `dict.get()`-en baliokidea

In [9]:
def h_get(h,k,d=None):
    x = h[hash(k) % len(h)]
    if x :
        k2,v2 = x
        if k2 == k :
            return v2
    return d 

In [10]:
print(h)
print(h_str(h))
print(f'hizt[3] = {h_get(h,3)}')
print(f'hizt[1] = {h_get(h,1)}')
print(f'hizt["lau"] = {h_get(h,"lau")}')
print(f'hizt[4] = {h_get(h,4)} , baina h[4]={h[4]}')

[None, None, ('lau', 4), (3, 'hiru'), None, None, None]
{ 'lau':4 , 3:'hiru' }
hizt[3] = hiru
hizt[1] = None
hizt["lau"] = 4
hizt[4] = None , baina h[4]=None


#### 7 - Hiztegi batetan gako bati dagokion balioa gordeko duen funtzioa, `h[k]=v`-en baliokidea.

In [11]:
def h_put(h,k,v):
    i = hash(k) % len(h)
    if h[i] :
        k2,v2 = h[i]
        if k2 == k :
            h[i] = (k,v)
        else :
            # KOLISIOA!!!!
            h_resize(h,int(len(h)*1.7))
            h_put(h,k,v)
    else :
        h[i] = (k,v)

`h_resize()` funtzio bat behar dugu...

#### 8 - Hiztegi baten *barne tamaina* aldatuko duen funtzioa

In [12]:
def h_resize(h,n):
    items = h_items(h)
    h.clear()
    h.extend([None] * n)
    for k,v in items :
        h_put(h,k,v)

`h_put()` eta `h_resize()` biak batera frogatuko ditugu...

In [13]:
h = h_new()
h_put(h,3,'hiru')
h_put(h,'lau',4)

print('-'*50+'\n',len(h),h)
print(h_str(h))

h_resize(h,9)
print('-'*50+'\n',len(h),h)
print(h_str(h))

h_resize(h,2)
print('-'*50+'\n',len(h),h)
print(h_str(h))
print(hash(3), hash('lau'))
print(hash(3)%10, hash('lau')%10)
print(hash(3)%17, hash('lau')%17)

--------------------------------------------------
 7 [None, None, ('lau', 4), (3, 'hiru'), None, None, None]
{ 'lau':4 , 3:'hiru' }
--------------------------------------------------
 9 [None, None, None, (3, 'hiru'), None, ('lau', 4), None, None, None]
{ 3:'hiru' , 'lau':4 }
--------------------------------------------------
 2 [('lau', 4), (3, 'hiru')]
{ 'lau':4 , 3:'hiru' }
3 5306439642013883732
3 2
3 10


#### 9 - Hiztegi baten *karga faktorea* kalkulatuko duen funtzioa

In [14]:
def h_lf(h):
    return h_len(h)/len(h)

Frogak egin ditzagun....

In [15]:
for n in 10,100,1000,10000 :
    h = h_new()
    for i in range(n):
        h_put(h,i,str(i))
    print(len(h),h_len(h),h_lf(h))

11 10 0.9090909090909091
146 100 0.684931506849315
1215 1000 0.823045267489712
10143 10000 0.9859016070196195


Zenbaki osoen hash balioa bere burua delako, `h` zerrendaren tamaina gordetzen ari garen `i` gako handiena baina handiagoa izatea nahikoa da... **EZ DA KOLISIORIK EGONGO**

In [16]:
for n in 10,100,1000,10000 :
    h = h_new()
    for i in range(n):
        h_put(h,str(i),i)
    print(len(h),h_len(h),h_lf(h))

146 10 0.0684931506849315
3510 100 0.02849002849002849
707534 1000 0.001413359640667445
49355783 10000 0.00020261050260310935


Lehenengo kasuan, gakoak `[0,n]` tarteko zenbakiak izan beharrean auzazkoak izan balira...

In [17]:
from random import randrange
for n in 10,100,1000,10000 :
    h = h_new()
    for i in range(n):
        h_put(h,randrange(1000000000),'KAKA!')
    print(len(h),h_len(h),h_lf(h))

30 10 0.3333333333333333
5967 100 0.01675884028825205
416197 1000 0.002402708332832769
29032814 10000 0.0003444378488423478
